# Python 中 NaN 和 None 的详细比较
http://python.jobbole.com/87266/

In [29]:
from numpy import NaN
from pandas import Series, DataFrame
import numpy as np
import pandas as pd

## 数据类型?
None是一个python特殊的数据类型， 但是NaN却是用一个特殊的float

In [5]:
type(None)

NoneType

In [6]:
type(NaN)

float

## 能作为dict的key?
都可以，而且会被认为是不同的key

In [7]:
{None:1}

{None: 1}

In [8]:
{NaN:1}

{nan: 1}

## Series函数中的表现
总结: 用Series.map对None进行替换时,会“顺便”把NaN也一起替换掉；NaN也会顺便把None替换掉。

如果None和NaN分别定义了不同的映射数值，那么只有NaN会生效。

In [10]:
s = Series([None, NaN, 'a'])
s

0    None
1     NaN
2       a
dtype: object

### 可以看到None和NaN都会替换成了1

In [11]:
s.map({None:1,'a':'a'})

0    1
1    1
2    a
dtype: object

### 同样None和NaN都会替换成了1

In [13]:
s.map({NaN:1,'a':'a'})

0    1
1    1
2    a
dtype: object

### 将None替换成1的要求被忽略了

In [16]:
s.map({NaN:2,'None':1,'a':'a'})

0    2
1    2
2    a
dtype: object

In [17]:
s.map({'None':1,NaN:2,'a':'a'})

0    2
1    2
2    a
dtype: object

## Series.replace中的表现
和Series.map的情况类似,指定了None的替换值后,NaN会被替换掉;反之亦然。


In [19]:
s1 = Series([None,NaN,'a'])
s1

0    None
1     NaN
2       a
dtype: object

In [22]:
s1.replace([NaN],9)

0    9
1    9
2    a
dtype: object

In [23]:
s1.replace([None],8)

0    8
1    8
2    a
dtype: object

## 对函数的支持

numpy有不少函数可以自动处理NaN。
pandas中也有不少函数支持NaN却不支持None。(毕竟pandas的底层是numpy)

In [26]:
np.nansum([1,2,NaN])

3.0

In [28]:
try:
    np.nansum([1,2,None])
except Exception as e:
    print(type(e),e)

<class 'TypeError'> unsupported operand type(s) for +: 'int' and 'NoneType'


In [39]:
pd.cut(np.array([.2, 1.4, 2.5, 6.2, 9.7, 2.1,NaN]), 3 ,labels=["good","medium","bad"])

[good, good, good, medium, bad, good, NaN]
Categories (3, object): [good < medium < bad]

## 对容器数据类型的影响

混入numpy.array的影响

如果数据中含有None,会导致整个array的类型变成object。

In [40]:
np.array([1]).dtype

dtype('int64')

In [41]:
np.array([1, None]).dtype

dtype('O')

In [42]:
np.array([1, NaN]).dtype

dtype('float64')

## 混入Series的影响
pandas将None自动替换成了NaN！
却是Object类型的None被替换成了float类型的NaN。 这么设计可能是因为None无法参与numpy的大多数计算， 而pandas的底层又依赖于numpy，因此做了这样的自动转化。

In [43]:
Series([1,NaN])

0    1.0
1    NaN
dtype: float64

In [44]:
Series([1,None])

0    1.0
1    NaN
dtype: float64

## 等值性判断

单值的等值性比较

In [45]:
None == None

True

In [46]:
NaN == NaN

False

In [47]:
None == NaN

False

In [49]:
#在tuple中的情况, 和基准不一致
(1, NaN) == (1, NaN)

True

In [51]:
np.array([1,None]) == np.array([1,None])

array([ True,  True], dtype=bool)

# TBD...

# Summary
None vs NaN要点总结

在pandas中， 如果其他的数据都是数值类型， pandas会把None自动替换成NaN, 甚至能将s[s.isnull()]= None,和s.replace(NaN, None)操作的效果无效化。 这时需要用where函数才能进行替换。
None能够直接被导入数据库作为空值处理， 包含NaN的数据导入时会报错。
numpy和pandas的很多函数能处理NaN，但是如果遇到None就会报错。
None和NaN都不能被pandas的groupby函数处理，包含None或者NaN的组都会被忽略。
等值性比较的总结:（True表示被判定为相等）

- , None对None, NaN对NaN, None对NaN
- 单值,     True, 	False, 	False
- tuple(整体), 	True, 	True, 	False
- np.array(逐个), 	True, 	False, 	False
- Series(逐个), 	False, 	False, 	False
- assert_equals, 	True, 	True, 	False
- Series.equals, 	True, 	True, 	True
- merge, 	True, 	True, 	True

由于等值性比较方面，None和NaN在各场景下表现不太一致，相对来说None表现的更稳定。

为了不给自己惹不必要的麻烦和额外的记忆负担。 实践中，建议遵循以下三个原则即可

1. 在用pandas和numpy处理数据阶段将None,NaN统一处理成NaN,以便支持更多的函数。
2. 如果要判断Series,numpy.array整体的等值性，用专门的Series.equals,numpy.array函数去处理，不要自己用==判断 *
3. 如果要将数据导入数据库，将NaN替换成None